In [ ]:
import matplotlib.pyplot as plt
from Bio.SeqIO.FastaIO import SimpleFastaParser
import mmh3
import sys, time
from collections import Counter
import sys, itertools
import multiprocessing as mp

import numpy as np
import matplotlib.pyplot as plt
import imp, os, pickle, time, tables
import scipy.sparse as sp_sparse
from sklearn.metrics.pairwise import pairwise_distances
import seaborn as sns
import scipy.stats
import csv, argparse
import subprocess, shutil
from tqdm import tqdm, trange
from sklearn.metrics import roc_curve, auc
import pickle

%matplotlib inline

In [ ]:
## path to dataset
# dataset = "/data/MAB_alignment/ecoli_maxSVD/"
dataset = "/data/MAB_alignment/NCTC4174_maxSVD/"

In [ ]:
## load minHashArr, created using mmh3 python hash library
minHashArr= np.loadtxt(dataset + "/minHashes/minHashArr.txt")
n,H = minHashArr.shape
print(n,H)

In [ ]:
## load ground truth alignments to compare with estimated u_i's
gt_file = dataset+'daligner_ground_truth.txt'
with open(gt_file) as f:
	lines = [[float(x) for x in line.rstrip('\n').split('\t')] for line in f]
refDict = {}
for i in range(n):
    refDict[i] = {}
for line in lines:
    refDict[int(line[0])-1][int(line[1])-1] = line[2]/(line[3]+line[4]-line[2]) 
    refDict[int(line[1])-1][int(line[0])-1] = line[2]/(line[3]+line[4]-line[2])

In [ ]:
### Adaptive Spectral Top-k Algorithm

def adaSH(A,budget,k):
    # Input:
        # binary minHash collision matrix A
        # maximum computational budget
        # number of reads to return k
    # Output: 
        # indices of top k reads
        # average number of minHashes used per read
            # this will be less than budget/n
    n,h = A.shape
    
    empReward = np.zeros(n)
    T = budget*n
    active = np.array(range(n))

    empReward = np.zeros(n)
    numPullsArr = np.zeros(n)
    numRounds = int(np.ceil(np.log2(n/k)))
    numPullsSoFar = 0
    
    ordering = np.random.permutation(h)
    for t in range(numRounds):
        numPulls = min(int(T/numRounds/len(active)),h)
        numPullsSoFar = min(numPullsSoFar+numPulls,h)
        empMatrix = A[np.ix_(active,ordering[:numPullsSoFar])]
        empQ = empMatrix.mean(axis=0)
        x = np.matmul(empMatrix-np.ones(empMatrix.shape),1-np.array(empQ/np.max(empQ)))

        empReward[active] = 1- np.abs(x)/np.abs(np.median(x))
        numPullsArr[active]= numPullsSoFar
    
        thresh = np.median(empReward[active])
        locs = np.where(empReward[active]>=thresh)
        active = active[locs]
        if numPulls == h:
            break
        if len(active) <= 2*k:
            break

    return active[np.argsort(-empReward[active])[:k]],numPullsArr.mean()

In [ ]:
ref_read = 2
numTrials = 1000
hs = np.logspace(np.log2(40),np.log2(10**3.8),num=20,endpoint=True,base=2) # target budget

k= len(list(refDict[ref_read].keys()))
print(hs)

unifSampling = np.zeros((len(hs),numTrials))
adaSampling = np.zeros((len(hs),numTrials))
adaBudget = np.zeros(len(hs))

empiricalMatrix = (minHashArr == minHashArr[ref_read])
empiricalMatrixBase = np.delete(empiricalMatrix,ref_read,axis=0)

groundTruthLocs = np.array(list(refDict[ref_read].keys()))
updatedgtLocs = groundTruthLocs - 1*(groundTruthLocs>=ref_read)

for i,h in tqdm(enumerate(hs), total=len(hs)):
    for t in range(numTrials):
        np.random.seed(t) ## for reproducibility
        
        ## run adaptive scheme
        arr,budget = adaSH(empiricalMatrixBase,h,k)
        adaSampling[i,t] = np.mean([1.0*(x in arr) for x in updatedgtLocs])
        adaBudget[i]=budget

        ## run uniform sampling scheme with same budget
        empiricalMatrix = empiricalMatrixBase[:,np.random.choice(H,size=int(budget)+1,replace=False)]
        empQ = empiricalMatrix.sum(axis=0)
        x = np.matmul(empiricalMatrix-np.ones(empiricalMatrix.shape),1-np.array(empQ/np.max(empQ)))
        x = 1- np.abs(x)/np.abs(np.median(x))
        
        ordering = np.argsort(-x)
        topK = ordering[:k]
        unifSampling[i,t] = np.mean([1.0*(x in topK) for x in updatedgtLocs])

In [ ]:
## generate additional points for uniform sampling
extraHs = [int(1.305**(i+1)*adaBudget[-1]) for i in range(2)]
print(extraHs)
numTrials = 1000
unifSamplingExtra = np.zeros((len(extraHs),numTrials))


empiricalMatrix = (minHashArr == minHashArr[ref_read])
empiricalMatrixBase = np.delete(empiricalMatrix,ref_read,axis=0)
groundTruthLocs = np.array(list(refDict[ref_read].keys()))
updatedgtLocs = groundTruthLocs - 1*(groundTruthLocs>=ref_read)
for i,h in enumerate(extraHs):
    for t in tqdm(range(numTrials),total=numTrials):
        np.random.seed(t)

        empiricalMatrix = empiricalMatrixBase[:,np.random.choice(H,size=h,replace=False)]
        empQ = empiricalMatrix.sum(axis=0)
        x = np.matmul(empiricalMatrix-np.ones(empiricalMatrix.shape),1-np.array(empQ/np.max(empQ)))
        x = 1- np.abs(x)/np.abs(np.median(x))
        
        ordering = np.argsort(-x)
        topK = ordering[:k]
        unifSamplingExtra[i,t] = np.mean([1.0*(x in topK) for x in updatedgtLocs])
    print( (unifSamplingExtra==1).mean(axis=1))

In [ ]:
# top-k error plot
pctiles = 5
fig, ax = plt.subplots(figsize=(5, 4))

unifBudget = adaBudget

CIunif = np.percentile(unifSampling, [pctiles/2, 100-pctiles/2], axis=1)
CIada = np.percentile(adaSampling, [pctiles/2, 100-pctiles/2], axis=1)

plt.style.use('ggplot')
plt.grid()

_,numSims = adaSampling.shape

plt.plot(adaBudget,adaSampling.mean(axis=1), "o-",label='Adaptive', linewidth=3)
ax.fill_between(adaBudget, CIada[0],CIada[1],
                color='red', alpha=0.3)

plt.plot(unifBudget,unifSampling.mean(axis=1),"o-", color="cornflowerblue", label='Non-adaptive', linewidth=3)
ax.fill_between(unifBudget, CIunif[0],CIunif[1],
                color='cornflowerblue', alpha=0.3)
plt.legend(loc="lower right")


plt.xscale('log')
plt.xlabel('Average number of hashes per read')
plt.ylabel('Fraction of top k recovered')
plt.title('Top-k alignment identification on ecoli')

In [ ]:
## waterfall plot

adaSampling_topK = 1-(adaSampling==1)
unifSampling_topK_new = 1-np.vstack((unifSampling==1,unifSamplingExtra==1))
adaBudgetPlot = adaBudget
adaBudgetPlotUnif = np.concatenate((adaBudget,extraHs))

fig, ax = plt.subplots(figsize=(5, 4))

plt.style.use('ggplot')
plt.xscale('log')


plt.plot(adaBudgetPlot,adaSampling_topK.mean(axis=1), "o-",label='Adaptive', linewidth=3)
ax.fill_between(adaBudgetPlot, adaSampling_topK.mean(axis=1) - numTrials**(-.5), 
                adaSampling_topK.mean(axis=1) + numTrials**(-.5), color='red', alpha=0.3)

plt.plot(adaBudgetPlotUnif,unifSampling_topK_new.mean(axis=1),"o-", color="cornflowerblue", label='Non-adaptive', linewidth=3)
ax.fill_between(adaBudgetPlotUnif, unifSampling_topK_new.mean(axis=1) - numTrials**(-.5), 
                unifSampling_topK_new.mean(axis=1) + numTrials**(-.5), color="cornflowerblue", alpha=0.3)
plt.legend(loc="lower left")



plt.xlabel('Average number of hashes per read')
plt.ylabel("Probability of Error")
plt.yscale('log')
plt.title('Top-k alignment identification on Ecoli')